In [2]:
import src.utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
stats_i = src.utils.json_load(snakemake.input[0])
_, v = next(iter(stats_i.items()))
v = pd.DataFrame.from_dict(v)
v = v.columns.to_list()
remover = ["stat", "up", "low"]
[v.remove(r) for r in remover]
# Make temp storage
percentage_fulfilled = {}
rmsd = {}
mae = {}
rmsd_stepwise = {}
# for different bundles
for i in v:
    percentage_fulfilled[i] = {}
    rmsd[i] = {}
    mae[i] = {}
    rmsd_stepwise[i] = {}

for i in snakemake.input:
    for j in v:
        stats = src.utils.json_load(i)
        hash = i[-31:-15]
        compound_id = i.split("/")[4]

        for k in stats.keys():
            stats_df = pd.DataFrame.from_dict(stats[k])
            percentage_fulfilled[j][f"{hash}-{compound_id}_{k}"] = stats_df[
                stats_df["stat"] == "percentage_fulfilled"
            ][j].values.tolist()[0]
            rmsd[j][f"{hash}-{compound_id}_{k}"] = stats_df[
                stats_df["stat"] == "RMSD"
            ][j].values.tolist()[0]
            mae[j][f"{hash}-{compound_id}_{k}"] = stats_df[
                stats_df["stat"] == "MAE"
            ][j].values.tolist()[0]
            rmsd_stepwise[j][f"{hash}-{compound_id}_{k}"] = stats_df[
                stats_df["stat"] == "RMSD_step"
            ][j].values.tolist()[0]
            


######
#         multiple = False
#         if "cis" in stats or "trans" in stats:
#             multiple = True
#         if multiple:
#             stats_cis = stats.cis
#             stats_trans = stats.trans
#             stats_cis = pd.DataFrame.from_dict(stats_cis)
#             stats_trans = pd.DataFrame.from_dict(stats_trans)
#         else:
#             test_NOE = pd.read_json(i)

#         if multiple:
#             if "stat" in stats_cis:
#                 percentage_fulfilled[f"{hash}-{compound_id}_cis"] = stats_cis[stats_cis['stat'] == 'percentage_fulfilled'].value.values.tolist()[0]
#                 rmsd[f"{hash}-{compound_id}_cis"] = stats_cis[stats_cis['stat'] == 'RMSD'].value.values.tolist()[0]
#                 mae[f"{hash}-{compound_id}_cis"] = stats_cis[stats_cis['stat'] == 'MAE'].value.values.tolist()[0]
#             if "stat" in stats_trans:
#                 percentage_fulfilled[f"{hash}-{compound_id}_trans"] = stats_trans[stats_trans['stat'] == 'percentage_fulfilled'].value.values.tolist()[0]
#                 rmsd[f"{hash}-{compound_id}_trans"] = stats_trans[stats_trans['stat'] == 'RMSD'].value.values.tolist()[0]
#                 mae[f"{hash}-{compound_id}_trans"] = stats_trans[stats_trans['stat'] == 'MAE'].value.values.tolist()[0]
#         else:
#             percentage_fulfilled[f"{hash}-{compound_id}"] = test_NOE[test_NOE['stat'] == 'percentage_fulfilled'].value.values.tolist()[0]
#             rmsd[f"{hash}-{compound_id}"] = test_NOE[test_NOE['stat'] == 'RMSD'].value.values.tolist()[0]
#             mae[f"{hash}-{compound_id}"] = test_NOE[test_NOE['stat'] == 'MAE'].value.values.tolist()[0]

In [6]:
stats_i

In [5]:
plt.scatter(
    percentage_fulfilled["value"].values(),
    percentage_fulfilled["value"].keys(),
)
plt.title(f"{snakemake.wildcards.method}: {snakemake.wildcards.simtime} ns")
plt.savefig(snakemake.output.plot)

In [6]:
output = {
    "RMSD": rmsd,
    "MAE": mae,
    "percentage_fulfilled": percentage_fulfilled,
    "RMSD_stepwise": rmsd_stepwise,
}

In [7]:
output

In [8]:
src.utils.json_dump(snakemake.output.data, output)